In [1]:
import numpy as np
from nose.tools import assert_almost_equal, assert_almost_equals, assert_equal

Ответами на задачи являются функции. Они будут проверены автоматическими тестами на стороне сервера. 

Некоторые тесты выполняются локально для самопроверки.

### Вопросы для самоконтроля 
Эта часть задания не оценивается, ответы можно не записывать
1. Что такое решающее дерево? Как по построенному дереву найти прогноз для объекта?
2. Почему для любой выборки можно построить дерево, имеющее нулевую ошибку на ней? Приведите примеры.
3. Почему не рекомендуется строить небинарные деревья (имеющие более двух потомков у каждой вершины)?
4. Как устроен жадный алгоритм построения дерева?
5. Какие критерии информативности для решения задачи классификации вы знаете?
6. Какой смысл у критерия Джини и энтропийного критерия?
7. Какие критерии информативности для решения задачи регрессии вы знаете?
8. Что такое pruning (стрижка) дерева? Чем отличаются post-pruning и pre-pruning?
9. Какие методы обработки пропущенных значений вы знаете?
10. Как учитывать категориальные признаки в решающем дереве?

### Критерии информативности (45%)

Критерий информативности для набора объектов $R$ вычисляется на основе того, насколько хорошо их целевые переменные предсказываются константой (при оптимальном выборе этой константы):
$$
H(R) = \min_{c \in Y} \dfrac{1}{|R|} \sum_{(x^i,y^i) \in R} L(y^i, c),
$$
где $L(y^i, c)$- некоторая функция потерь. Соответственно, чтобы получить вид критерия при конкретной функции потерь, можно аналитически найти оптимальное значение константы и подставить его в формулу для $H(R)$. 


Выведите критерии информативности для следующих функций потерь:

Для задачи регрессии,
1. $L(y,c) = (y-c)^2$, где $y$ - скаляр, c - константа.

Для задачи классификации на $K$ классов, с дополнительным ограничением
$$c = [c_1,\ldots,c_k], 0 \leq c_i \leq 1 \forall i, \sum_{k=1}^K c_k = 1,$$
2. $L(y,c) = \sum_{k=1}^K (c_k-[y_k=1])^2$,  где $y$ - это one-hot вектор, $y_k$ - его элемент k-тый элемент, $c$ - вектор вероятностей.
3. $L(y,c) = -\sum_{k=1}^K [y_k=1]\log c_k$,  где $y$ - это one-hot вектор, $y_k$ - его элемент k-тый элемент, $c$ - вектор вероятностей.

In [3]:
def H_1(ys):
    """
    ys is a 1-dimentional numpy array containing y values for every object from R.
    """
    # YOUR CODE HERE
    return np.sum((ys - ys.mean()) ** 2) / ys.shape[0]
   # raise NotImplementedError()

In [47]:
def H_2(ys):
    """
    ys is a numpy array with shape (num_items, num_classes).
    Where each row is a one-vector of class probabilities (e.g. [0, 0, 1] for object of class 2 from 0, 1, 2).
    """
    probs = (np.sum(ys, axis = 0)) / ys.shape[0]
    return np.sum((ys - probs) ** 2) / ys.shape[0]
    #raise NotImplementedError()

In [50]:
epsilon = 1e-5
def H_3(ys):
    """
    ys is a numpy array with shape (num_items, num_classes).
    Where each row is a one-vector of class probabilities (e.g. [0, 0, 1] for object of class 2 from 0, 1, 2).
    log2 should be used as logarithm. 
    Do not forget to add epsilon to the probabitlities vector in the logarithm.
    """
    probs = ((np.sum(ys, axis = 0)) / ys.shape[0]) + epsilon
    return - np.sum((np.sum(ys, axis = 0)) * np.log2(probs)) / ys.shape[0]
    #raise NotImplementedError()

In [51]:
a_r = np.arange(10)
b_r = np.ones(10)
c_r = np.arange(25)/10.

In [52]:
assert_equal(H_1(a_r), 8.25)
assert_equal(H_1(b_r), 0.0)
assert_equal(H_1(c_r), 0.52)

In [53]:
a = np.vstack((np.ones(10), np.zeros(10))).T
b = np.hstack([np.vstack((np.ones(5), np.zeros(5))), np.vstack((np.zeros(5), np.ones(5)))]).T
c = np.hstack([np.vstack((np.ones(9), np.zeros(9))), np.vstack((np.zeros(1), np.ones(1)))]).T
print('a:\n{}\nb:\n{}\nc:\n{}'.format(a, b, c))

a:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
b:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
c:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [54]:
assert_almost_equal(H_2(a), 0.0, places=4)
assert_almost_equal(H_2(b), 0.5, places=4)
assert_almost_equal(H_2(c), 0.18, places=4)

In [55]:
assert_almost_equal(H_3(a), 0.0, places=4)
assert_almost_equal(H_3(b), 1.0, places=4)
assert_almost_equal(H_3(c), 0.469, places=3)

### Сложность дерева (15%)

Запишите оценку сложности построения одного решающего дерева в зависимости от размера обучающей выборки $l$, числа признаков $d$, максимальной глубины дерева $D$. В качестве предикатов используются пороговые функции $[x_j>t]$. При выборе предиката в каждой вершине перебираются все признаки, а в качестве порогов рассматриваются величины $t$, равные значениям этого признака на объектах, попавших в текущую вершину. Считайте сложность вычисления критерия информативности на подвыборке константной (т.е. $O(1)$).

Оценку сложности представьте в формате $O($`get_tree_complexity(D, l, d)`$)$, где `get_tree_complexity` - некоторая функция от $D$, $l$ и $d$. Функцию реализуйте ниже. 

Пример использования (числа и зависимости случайны):
```
def get_tree_complexity(D, l, d):
    return D+l+d
    
a = get_tree_complexity(1, 2, 3)
```
Тогда число a == 6.

In [64]:
def get_tree_complexity(D, l, d):
    """
    Compute tree complexity in form O("some_expression") and return the "some_expression".
    """
#     if (l < 2 ** D):
#         return d * ((int)(np.log2(l + 1)))
#     else:
    return  D * l * d
   # raise NotImplementedError()

In [ ]:
#This cell is executed on the server side.


### Bootstrap (40%)

В данной задаче необходимо вычислить вероятность попадания объекта в boostrap-выборку, а затем оценить ее численно.


Пусть выборка $\hat{X}^{n}$ размера $n$ сгененирована методом bootstrap на основе выборки $X^{n}={\boldsymbol{x}_{1},\dots\boldsymbol{x}_{n}}$. Найдите вероятность попадания объекта $x_{i}$ в выборку $\hat{X}^{n}$ и вычислите ее для случая $n\rightarrow\infty$. Реализуйте функцию `probability_to_get_into_X_b`, которая возвращает эту вероятность как число от `0` до `1`. В качесте экспоненты можете использовать `math.exp(1)`.

In [23]:
import math

def probability_to_get_into_X_b():
    # YOUR CODE HERE
    return 1 - math.exp(-1)

In [24]:
assert_almost_equal(probability_to_get_into_X_b(), 0.6, places=1)

Реализуйте свою функцию, генерирующую bootstrap-выборку из исходной.

In [60]:
from scipy.stats import randint

def my_bootstrap(X):
    """
    Implement the function that returns the 
    bootstraped dataset of the same size the
    original dataset was.
    """
    rand_index = randint.rvs(0, X.shape[0], size = X.shape[0])
    return X[rand_index]
#     raise NotImplementedError()

Численно оцените вероятность попадания объекта исходной выборки в bootstrap-выборку для размера выборки `N`. Функция `get_sample_proba` должна возвращать число от `0` до `1`. 

Не забывайте, что мы живем в случайном мире ;)

In [61]:
def get_sample_proba(N):
    # YOUR CODE HERE
    #raise NotImplementedError()
    count_of_experiment = N * N
    luck = np.sum(list(map(lambda ar: np.any(my_bootstrap(np.arange(0, N)) == 0), np.arange(0, count_of_experiment))))
    return luck / count_of_experiment

In [ ]:
#This cell is executed on the server side.


Поздравляем, задание завершено. Не забудьте остановить свой виртуальный инстанс перед уходом (Control Panel -> Stop My Server).